In [130]:
from utils import load

In [131]:
data = load('day16.txt')
data = list(bin(int(data[0], 16))[2:].zfill(len(data[0]) * 4))

In [132]:
def parse_packet(packet):
    version = int(''.join(packet[:3]), 2)
    packet[:] = packet[3:]
    type_id = int(''.join(packet[:3]), 2)
    packet[:] = packet[3:]
    
    if type_id == 4:
        literal = []
        while True:
            I = packet.pop(0)
            literal += packet[:4]
            packet[:] = packet[4:]
            if I == '0':
                break
        literal = int(''.join(literal), 2)
        return (version, type_id, literal)
    else:
        packets = []
        length_type_id = packet.pop(0)
        if length_type_id == '0':
            subpackets_len = int(''.join(packet[:15]), 2)
            packet[:] = packet[15:]
            subpackets = packet[:subpackets_len]
            packet[:] = packet[subpackets_len:]
            while subpackets:
                packets.append(parse_packet(subpackets))
        else:
            subpackets_num = int(''.join(packet[:11]), 2)
            packet[:] = packet[11:]
            for i in range(subpackets_num):
                packets.append(parse_packet(packet))
        return (version, type_id, packets)

In [133]:
parsed = parse_packet(data)

In [134]:
def vsum(data):
    version = data[0]
    if data[1] == 4:
        return version
    else:
        return version + sum(map(vsum, data[2]))

In [135]:
vsum(parsed)

938

In [136]:
def calc(data):
    if data[1] == 0:
        return sum(map(calc, data[2]))
    elif data[1] == 1:
        t = 1
        for payload in data[2]:
            t *= calc(payload)
        return t
    elif data[1] == 2:
        return min(map(calc, data[2]))
    elif data[1] == 3:
        return max(map(calc, data[2]))
    elif data[1] == 4:
        return data[2]
    elif data[1] == 5:
        return calc(data[2][0]) > calc(data[2][1])
    elif data[1] == 6:
        return calc(data[2][0]) < calc(data[2][1])
    elif data[1] == 7:
        return calc(data[2][0]) == calc(data[2][1])

In [137]:
calc(parsed)

1495959086337